In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
# Load dataset
mnist = input_data.read_data_sets('./dataset/MNIST_data',one_hot=True)

# param set
batch_size = 2000                                  #单批次数据量
n_batch = mnist.train.num_examples // batch_size  #批次数量
hidden_layer1_node = 500                         #隐藏层1节点数
hidden_layer2_node = 300                         #隐藏层2节点数
output_layer_node = 10                            #输出层节点数
steps = 3000                                        #训练迭代次数
lr = 0.3                                          #学习速率
regularization_rate = 0.0015                       #正则化系数
image_num = 300

embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

# 训练数据插入点
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,output_layer_node])
    keep_prob = tf.placeholder(tf.float32)

# display image

image_shaped_input = tf.reshape(x,[-1,28,28,1])
tf.summary.image('input',image_shaped_input,10)
    
def variable_summaries(var, name):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean/' + name, mean)  
        with tf.name_scope('stddev'):  
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))  
        tf.summary.scalar('sttdev/' + name, stddev)  
        tf.summary.scalar('max/' + name, tf.reduce_max(var))  
        tf.summary.scalar('min/' + name, tf.reduce_min(var))  
        tf.summary.histogram(name, var) 
        
def inference(input_tensor,hidden_layer1_node,hidden_layer2_node,
              keep_prob,regularization_rate,reuse=False):
# weight & biases
    with tf.variable_scope('hidden_layer1',reuse=reuse):
        hidden_layer1_w = tf.get_variable('w',[784,hidden_layer1_node],initializer=tf.truncated_normal_initializer(stddev=0.1))
        variable_summaries(hidden_layer1_w, 'hidden_layer1_w')
        hidden_layer1_b = tf.get_variable('b',hidden_layer1_node,
                                          initializer=tf.constant_initializer(0.1))
        variable_summaries(hidden_layer1_b, 'hidden_layer1_b')
        hidden_layer1 = tf.matmul(x,hidden_layer1_w) + hidden_layer1_b
        hidden_layer1 = tf.nn.relu(hidden_layer1)
        hidden_layer1 = tf.nn.dropout(x=hidden_layer1,keep_prob=keep_prob)
        
    with tf.variable_scope('hidden_layer2',reuse=reuse):
        hidden_layer2_w = tf.get_variable('w',[hidden_layer1_node,hidden_layer2_node],
                                           initializer=tf.truncated_normal_initializer(stddev=0.1))
        variable_summaries(hidden_layer2_w, 'hidden_layer2_w')
        hidden_layer2_b = tf.get_variable('b',hidden_layer2_node,
                                           initializer=tf.constant_initializer(0.1))
        variable_summaries(hidden_layer2_b, 'hidden_layer2_b')
        hidden_layer2 = tf.matmul(hidden_layer1,hidden_layer2_w) + hidden_layer2_b
        hidden_layer2 = tf.nn.relu(hidden_layer2)
        hidden_layer2 = tf.nn.dropout(x=hidden_layer2,keep_prob=keep_prob)
        
    with tf.variable_scope('out_layer',reuse=reuse):
        out_layer_w = tf.get_variable('w',[hidden_layer2_node,output_layer_node],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        variable_summaries(out_layer_w, 'out_layer_w')
        out_layer_b = tf.get_variable('b',output_layer_node,
                                      initializer=tf.constant_initializer(0.1))
        variable_summaries(out_layer_b, 'out_layer_b')
        prediction = tf.matmul(hidden_layer2,out_layer_w) + out_layer_b
        
    with tf.variable_scope('regular',reuse=reuse):
        regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)
        regularization = regularizer(hidden_layer1_w) + regularizer(hidden_layer2_w) + regularizer(out_layer_w)
        variable_summaries(regularization, 'regularization')
    return prediction,regularization

prediction,regularization = inference(x,hidden_layer1_node,hidden_layer2_node,keep_prob,regularization_rate) 

# 损失函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction)) + regularization
    tf.summary.scalar('loss', loss)

# 定义训练方式
with tf.name_scope('train_step'):
    train_step = tf.train.AdadeltaOptimizer(lr).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#准确率
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy', accuracy)

# runtime
with tf.Session() as sess:
    #初始化变量
    sess.run(init)
    
    #write metadata
    if tf.gfile.Exists('./tensorborad/logs/metadata.tsv'):
        tf.gfile.DeleteRecursively('./tensorborad/logs/metadata.tsv')
    with open('./tensorborad/logs/metadata.tsv','w') as f:
        labels = sess.run(tf.argmax(mnist.test.labels[:],1))
        for i in range(image_num):
            f.write(str(labels[i]) + '\n')

    #merged summary
    merged = tf.summary.merge_all() 
    
    writer = tf.summary.FileWriter('./tensorborad/logs',tf.get_default_graph())
    saver = tf.train.Saver()
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = embedding.name
    embed.metadata_path = './tensorborad/logs/metadata.tsv'
    embed.sprite.image_path = './tensorborad/logs/mnist_10k_sprite.png'
    embed.sprite.single_image_dim.extend([28,28])
    projector.visualize_embeddings(writer,config)
    
    train_writer = tf.summary.FileWriter('./tensorborad/logs/train')
    test_writer = tf.summary.FileWriter('./tensorborad/logs/test')
    for i in range(steps):
        x_train,y_train = mnist.train.next_batch(i)
        sess.run(train_step,feed_dict={x:x_train,y:y_train,keep_prob:0.8})
        if (i+1) % 500 ==0:
            summary, acc = sess.run([merged,accuracy],feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            train_writer.add_summary(summary, i) 
            summary,train_acc = sess.run([merged,accuracy],feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
            test_writer.add_summary(summary, i) 
            print('Iter %d, Testing Accuracy: %f,Train Accuracy:%f' % (i+1,acc,train_acc))
            saver.save(sess,'./tensorborad/logs/model.ckpt',i)
    
train_writer.close()
test_writer.close()

Extracting ./dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 500, Testing Accuracy: 0.919200,Train Accuracy:0.915091
Iter 1000, Testing Accuracy: 0.951500,Train Accuracy:0.950745
Iter 1500, Testing Accuracy: 0.963100,Train Accuracy:0.965818
Iter 2000, Testing Accuracy: 0.970500,Train Accuracy:0.974400
Iter 2500, Testing Accuracy: 0.973400,Train Accuracy:0.979109
Iter 3000, Testing Accuracy: 0.975100,Train Accuracy:0.982873


In [15]:
from hyperopt import hp, fmin, rand, tpe, space_eval, partial

In [6]:
hidden_layer1_node = 500                         #隐藏层1节点数
hidden_layer2_node = 300                         #隐藏层2节点数
lr = 0.3                                          #学习速率
regularization_rate = 0.0015                       #正则化系数

In [7]:
space = [hp.randint('hidden_layer1_node',10,1000),
         hp.randint('hidden_layer2_node',10,1000),
         hp.normal('lr',0,1),
         hp.normal('regularization_rate',0,0.01),
         hp.normal('keep_prob',0,1)]

In [8]:
space

In [9]:
from hyperopt.pyll.stochastic import sample

In [13]:
print sample(space)

SyntaxError: invalid syntax (<ipython-input-13-2577f7411c47>, line 1)

In [19]:
def percept(args):
    global X_train_std,y_train,y_test
    ppn,regularization = inference(input_tensor=x,hidden_layer1_node = int(args['hidden_layer1_node']),
                                   hidden_layer2_node= int(args[hidden_layer2_node]),
                                   keep_prob = args['keep_prob'],
                                   regularization_rate = args['regularization_rate'],
                                   reuse=False)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=ppn)) + regularization
    train_step = tf.train.AdadeltaOptimizer(args['lr']).minimize(loss)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    return -accuracy

In [21]:
def NN(argsDict):
    hidden_layer1_node = argsDict['hidden_layer1_node']                         #隐藏层1节点数
    hidden_layer2_node = argsDict['hidden_layer2_node']                         #隐藏层2节点数
    lr = argsDict['lr']                                                         #学习速率
    regularization_rate = argsDict['regularization_rate']                       #正则化系数
    keep_prob = argsDict['keep_prob']                                           #dropout系数
    print 'hidden_layer1_node:' + str(hidden_layer1_node)
    print 'hidden_layer2_node:' + str(hidden_layer2_node)
    print 'lr' + str(lr)
    print 'regularization_rate' + str(regularization_rate)
    print 'keep_prob' + str(keep_prob)
    global attr_train,label_train
    
    nn = train_nn(hidden_layer1_node,hidden_layer2_node,keep_prob,regularization_rate)
    loss =

TypeError: ap_categorical_sampler() got multiple values for argument 'size'

In [ ]:
batch_size = 1000                                  #单批次数据量
n_batch = mnist.train.num_examples // batch_size  #批次数量
hidden_layer1_node = 500                         #隐藏层1节点数
hidden_layer2_node = 300                         #隐藏层2节点数
output_layer_node = 10                            #输出层节点数
steps = 3000                                        #训练迭代次数
lr = 0.3                                          #学习速率
regularization_rate = 0.0015                       #正则化系数
image_num = 10000
def train_nn(hidden_layer1_node,hidden_layer2_node,keep_prob,regularization_rate):
    def inference(input_tensor,hidden_layer1_node,hidden_layer2_node,
                  keep_prob,regularization_rate,reuse=False):
    # weight & biases
        with tf.variable_scope('hidden_layer1',reuse=reuse):
            hidden_layer1_w = tf.get_variable('w',[784,hidden_layer1_node],initializer=tf.truncated_normal_initializer(stddev=0.1))
            variable_summaries(hidden_layer1_w, 'hidden_layer1_w')
            hidden_layer1_b = tf.get_variable('b',hidden_layer1_node,
                                              initializer=tf.constant_initializer(0.1))
            variable_summaries(hidden_layer1_b, 'hidden_layer1_b')
            hidden_layer1 = tf.nn.relu(tf.matmul(x,hidden_layer1_w) + hidden_layer1_b)
            hidden_layer1 = tf.nn.dropout(x=hidden_layer1,keep_prob=keep_prob)

        with tf.variable_scope('hidden_layer2',reuse=reuse):
            hidden_layer2_w = tf.get_variable('w',[hidden_layer1_node,hidden_layer2_node],
                                               initializer=tf.truncated_normal_initializer(stddev=0.1))
            variable_summaries(hidden_layer2_w, 'hidden_layer2_w')
            hidden_layer2_b = tf.get_variable('b',hidden_layer2_node,
                                               initializer=tf.constant_initializer(0.1))
            variable_summaries(hidden_layer2_b, 'hidden_layer2_b')
            hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1,hidden_layer2_w) + hidden_layer2_b)
            hidden_layer2 = tf.nn.dropout(x=hidden_layer2,keep_prob=keep_prob)

        with tf.variable_scope('out_layer',reuse=reuse):
            out_layer_w = tf.get_variable('w',[hidden_layer2_node,output_layer_node],
                                          initializer=tf.truncated_normal_initializer(stddev=0.1))
            variable_summaries(out_layer_w, 'out_layer_w')
            out_layer_b = tf.get_variable('b',output_layer_node,
                                          initializer=tf.constant_initializer(0.1))
            variable_summaries(out_layer_b, 'out_layer_b')
            prediction = tf.matmul(hidden_layer2,out_layer_w) + out_layer_b
        with tf.variable_scope('regular',reuse=reuse):
            regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)
            regularization = regularizer(hidden_layer1_w) + regularizer(hidden_layer2_w) + regularizer(out_layer_w)
            variable_summaries(regularization, 'regularization')
        return prediction,regularization

    prediction,regularization = inference(x,keep_prob,regularization_rate) 

    # 损失函数
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction)) + regularization
        tf.summary.scalar('loss', loss)

    # 定义训练方式
    with tf.name_scope('train_step'):
        train_step = tf.train.AdadeltaOptimizer(lr).minimize(loss)

    #初始化变量
    init = tf.global_variables_initializer()

    #准确率
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy', accuracy)

    # runtime
    with tf.Session() as sess:
        #初始化变量
        sess.run(init)

        #write metadata
        if tf.gfile.Exists('./tensorborad/logs/metadata.tsv'):
            tf.gfile.DeleteRecursively('./tensorborad/logs/metadata.tsv')
        with open('./tensorborad/logs/metadata.tsv','w') as f:
            labels = sess.run(tf.argmax(mnist.test.labels[:],1))
            for i in range(image_num):
                f.write(str(labels[i]) + '\n')

        #merged summary
        merged = tf.summary.merge_all() 

        writer = tf.summary.FileWriter('./tensorborad/logs',tf.get_default_graph())
        saver = tf.train.Saver()
        config = projector.ProjectorConfig()
        embed = config.embeddings.add()
        embed.tensor_name = embedding.name
        #embed.metadata_path = './tensorborad/logs/metadata.tsv'
        #embed.sprite.image_path = './tensorborad/logs/mnist_10k_sprite.png'
        embed.sprite.single_image_dim.extend([28,28])
        projector.visualize_embeddings(writer,config)

        train_writer = tf.summary.FileWriter('./tensorborad/logs/train')
        test_writer = tf.summary.FileWriter('./tensorborad/logs/test')
        for i in range(steps):
            x_train,y_train = mnist.train.next_batch(i)
            sess.run(train_step,feed_dict={x:x_train,y:y_train,keep_prob:0.8})
            if (i+1) % 50 ==0:
                summary, acc = sess.run([merged,accuracy],feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
                train_writer.add_summary(summary, i) 
                summary,train_acc = sess.run([merged,accuracy],feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
                test_writer.add_summary(summary, i) 
                print('Iter %d, Testing Accuracy: %f,Train Accuracy:%f' % (i+1,acc,train_acc))
                saver.save(sess,'./tensorborad/logs/model.ckpt',i)
        return loss